In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
#pre-processed data
df_n_des = pd.read_csv('Data/restaurant_review.csv')
df_n_des = df_n_des[df_n_des.user_id != '#NAME?']
df_n_des.tail()

,name,business_id,categories,is_closed,review_id,review_rating,review_text,user_id
2851,res_6,a6,"Asian Fusion,Thai,Sports Bars",NaN,NaN,NaN,NaN,NaN
2852,res_7,a7,"Pubs,German,Beer Bar",NaN,NaN,NaN,NaN,NaN
2853,res_8,a8,"Asian Fusion,Ramen,Noodles",NaN,NaN,NaN,NaN,NaN
2854,res_9,a9,"Indian,Himalayan/Nepalese,Vegan",NaN,NaN,NaN,NaN,NaN
2855,res_10,a10,"Sushi Bars,Japanese",NaN,NaN,NaN,NaN,NaN


In [80]:
#Keep only required columns
df_n_des_1 = df_n_des[['name','business_id','categories']]
df_n_des_1.tail()

,name,business_id,categories
2851,res_6,a6,"Asian Fusion,Thai,Sports Bars"
2852,res_7,a7,"Pubs,German,Beer Bar"
2853,res_8,a8,"Asian Fusion,Ramen,Noodles"
2854,res_9,a9,"Indian,Himalayan/Nepalese,Vegan"
2855,res_10,a10,"Sushi Bars,Japanese"


In [81]:
#remove duplicates and keep only unique values of business ids
res_wd = df_n_des_1[df_n_des['categories'].notnull()].copy()
res_wd = res_wd [['name','business_id','categories']]
res_wd = res_wd.drop_duplicates()
res_wd = res_wd.reset_index(drop=True)
res_wd.tail()

,name,business_id,categories
954,res_6,a6,"Asian Fusion,Thai,Sports Bars"
955,res_7,a7,"Pubs,German,Beer Bar"
956,res_8,a8,"Asian Fusion,Ramen,Noodles"
957,res_9,a9,"Indian,Himalayan/Nepalese,Vegan"
958,res_10,a10,"Sushi Bars,Japanese"


In [82]:
#To get keywords from categories
df_item = res_wd[['business_id','categories']]
df_item.tail()

,business_id,categories
954,a6,"Asian Fusion,Thai,Sports Bars"
955,a7,"Pubs,German,Beer Bar"
956,a8,"Asian Fusion,Ramen,Noodles"
957,a9,"Indian,Himalayan/Nepalese,Vegan"
958,a10,"Sushi Bars,Japanese"


In [4]:
#one hot matrix --> df_item
df_item = pd.read_csv('Data/business_category_mapping.csv')
df_item = df_item.rename(columns={"Unnamed: 0": "business_id"})
df_item.set_index('business_id', inplace=True)
df_item.tail()

,Wine & Spirits,Acai Bowls,Afghan,American (New),American (Traditional),Arabian,Arcades,Argentine,Armenian,Asian Fusion,...,Vegan,Vegetarian,Venezuelan,Venues & Event Spaces,Vermouth Bars,Vietnamese,Waffles,Whiskey Bars,Wine Bars,Wine Tasting Room
business_id,,,,,,,,,,,,,,,,,,,,,
a6,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
a7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a8,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
a9,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
a10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
df_merged = df_n_des[['user_id','business_id','review_rating','name','categories']]
df_merged.head()

,user_id,business_id,review_rating,name,categories
0,3UxhJ0zdww8ptAESBMwAaQ,f-m7-hyFzkf0HSEeQ2s-9A,5.0,Fog Harbor Fish House,"Seafood,Bars"
1,XoPZFbhk60YaXOlFvfBFIA,f-m7-hyFzkf0HSEeQ2s-9A,3.0,Fog Harbor Fish House,"Seafood,Bars"
2,iq9b1xt3PG1rBbjGY4I6AQ,f-m7-hyFzkf0HSEeQ2s-9A,1.0,Fog Harbor Fish House,"Seafood,Bars"
3,ZDLnBs5w9Lh_j2t9fBLRLA,HHtpR0RslupSQ99GIIwW5A,5.0,Marufuku Ramen SF,Ramen
5,zNtNVYxpMGPOviQK-YUbEg,HHtpR0RslupSQ99GIIwW5A,4.0,Marufuku Ramen SF,Ramen


In [86]:
df_final = df_merged[df_merged['business_id'].isin(df_item.index)]


In [87]:
#create business_id and category dict
bid_cat_dict = pd.Series(res_wd['categories'].values,index=res_wd['business_id']).to_dict()

In [88]:
df_final['categories'] = df_final['business_id'].map(bid_cat_dict)


In [89]:
#User review data and business data
df_final.head()

,user_id,business_id,review_rating,name,categories
0,3UxhJ0zdww8ptAESBMwAaQ,f-m7-hyFzkf0HSEeQ2s-9A,5.0,Fog Harbor Fish House,"Seafood,Bars"
1,XoPZFbhk60YaXOlFvfBFIA,f-m7-hyFzkf0HSEeQ2s-9A,3.0,Fog Harbor Fish House,"Seafood,Bars"
2,iq9b1xt3PG1rBbjGY4I6AQ,f-m7-hyFzkf0HSEeQ2s-9A,1.0,Fog Harbor Fish House,"Seafood,Bars"
3,ZDLnBs5w9Lh_j2t9fBLRLA,HHtpR0RslupSQ99GIIwW5A,5.0,Marufuku Ramen SF,Ramen
5,zNtNVYxpMGPOviQK-YUbEg,HHtpR0RslupSQ99GIIwW5A,4.0,Marufuku Ramen SF,Ramen


In [90]:
df_final = df_final[['user_id','review_rating','business_id','name','categories']]
user_ratings = df_final.user_id.value_counts()

In [91]:
rating = pd.pivot_table(df_final, values='review_rating', index=['business_id'], columns = ['user_id'])
rating.sort_index(axis=1, inplace=True)
rating.head()

user_id,--5MV_3EqmQinXwJPNI2fg,-2MQImZe-NXRIEg1ACLjaw,-7QPLtnJ0lEBi-ChP3betw,-7cPERHtAupd1Xq53nmhXg,-8hnmBOU3e3HfK2QIzsTCA,-9X63yvBKJyOaQzP-Clvkw,-9d1JcDTLGZAI2Jgg58snA,0-B5KklGJThDsJqywFROXQ,0-LBHFOdDyutdJ91ca_9Lg,02iSxqxrLAitketfCcuXWA,...,zivzUCqNsFfwpD8Rqmyqtw,zjunK1Kuxk1Ov5uuR4JOTg,zlLIWeAAPz_UKlHFJwbzPA,zn2X-YYbk2OdnpznDvFRvg,zoahf1GKypVKEe1GxH0OUw,zsd6KYwlOAKNjDx3_KdV_g,ztEXpU2gQkgHvuhZiyQE2w,ztVCqx-qqF1HMxl5McgUzw,zu_w6uXGCTbhouKHfDVIFg,zux1bck92ONBnvuLsVLUpQ
business_id,,,,,,,,,,,,,,,,,,,,,
#NAME?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-4wz0dOG3RdBNYE02l9KjA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01BcGT2SkIh2t95_M4Tg7w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
055dTcfz2AOAFPed_2KjEQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0QydZwQebF1b36i9OwHppA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
users_no = rating.columns
df_users = pd.DataFrame(columns = df_item.columns)
df_users

,Wine & Spirits,Acai Bowls,Afghan,American (New),American (Traditional),Arabian,Arcades,Argentine,Armenian,Asian Fusion,...,Vegan,Vegetarian,Venezuelan,Venues & Event Spaces,Vermouth Bars,Vietnamese,Waffles,Whiskey Bars,Wine Bars,Wine Tasting Room


In [93]:
for i in tqdm(range(len(users_no))):
    working_df = df_item.mul(rating.iloc[:,i], axis=0)
    working_df.replace(0, np.NaN, inplace=True)    
    df_users.loc[users_no[i]] = working_df.mean(axis=0)

100%|██████████| 2251/2251 [00:09<00:00, 229.50it/s]


In [94]:
df_users.head()

,Wine & Spirits,Acai Bowls,Afghan,American (New),American (Traditional),Arabian,Arcades,Argentine,Armenian,Asian Fusion,...,Vegan,Vegetarian,Venezuelan,Venues & Event Spaces,Vermouth Bars,Vietnamese,Waffles,Whiskey Bars,Wine Bars,Wine Tasting Room
--5MV_3EqmQinXwJPNI2fg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2MQImZe-NXRIEg1ACLjaw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN
-7QPLtnJ0lEBi-ChP3betw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-7cPERHtAupd1Xq53nmhXg,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-8hnmBOU3e3HfK2QIzsTCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
document_frequency = df_item.sum()

In [96]:
idf = (len(res_wd)/document_frequency).apply(np.log)
idf_df_item = df_item.mul(idf.values)

In [97]:
df_predict = pd.DataFrame()

In [98]:
for i in tqdm(range(len(users_no))):
    working_df = idf_df_item.mul(df_users.iloc[i], axis=1)
    df_predict[users_no[i]] = working_df.sum(axis=1)

100%|██████████| 2251/2251 [01:36<00:00, 23.43it/s]


In [99]:
df_predict.tail()

,--5MV_3EqmQinXwJPNI2fg,-2MQImZe-NXRIEg1ACLjaw,-7QPLtnJ0lEBi-ChP3betw,-7cPERHtAupd1Xq53nmhXg,-8hnmBOU3e3HfK2QIzsTCA,-9X63yvBKJyOaQzP-Clvkw,-9d1JcDTLGZAI2Jgg58snA,0-B5KklGJThDsJqywFROXQ,0-LBHFOdDyutdJ91ca_9Lg,02iSxqxrLAitketfCcuXWA,...,zivzUCqNsFfwpD8Rqmyqtw,zjunK1Kuxk1Ov5uuR4JOTg,zlLIWeAAPz_UKlHFJwbzPA,zn2X-YYbk2OdnpznDvFRvg,zoahf1GKypVKEe1GxH0OUw,zsd6KYwlOAKNjDx3_KdV_g,ztEXpU2gQkgHvuhZiyQE2w,ztVCqx-qqF1HMxl5McgUzw,zu_w6uXGCTbhouKHfDVIFg,zux1bck92ONBnvuLsVLUpQ
business_id,,,,,,,,,,,,,,,,,,,,,
a6,0.0,0.0,0.0,0.000000,2.974071,0.000000,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,18.769329,0.0,0.0,11.896283
a7,0.0,0.0,0.0,16.631363,0.000000,0.000000,0.0,0.0,0.0,22.81653,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
a8,0.0,0.0,0.0,0.000000,2.974071,11.896283,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,14.870354,31.423069,8.922212,0.0,0.0,11.896283
a9,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,23.932248
a10,0.0,0.0,0.0,0.000000,12.722164,0.000000,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,16.141525


In [100]:
def recommender_restaurants_to_user(user_no):
    
    #get all res bus id
    b_id_no = df_predict.index

    #user predicted rating to all restaurants
    user_predicted_rating = df_predict[user_no]

    user_rating_res =  pd.merge(user_predicted_rating, res_wd, how ='inner', on ='business_id')
    user_rating_res = user_rating_res.set_index('business_id')
    
    #restaurants already visited by user
    already_visited = df_final[df_final['user_id'].isin([user_no])]['business_id']
    
    #recommendation without restaurants being visited by user
    all_rec = user_rating_res[~user_rating_res.index.isin(already_visited)]

    return all_rec.sort_values(by=[user_no], ascending=False).iloc[0:10]

In [101]:
#List of business id that will be recommended to user_no
recommender_restaurants_to_user('B-tnUR66MKSoLtzMrHBywA')

,B-tnUR66MKSoLtzMrHBywA,name,categories
business_id,,,
QDb7LJMMJ9J4zwRC1LFRdg,37.916569,Bussaba,"Thai,Gluten-Free,Comfort Food"
AE2ong5QiDhG5bz4vQnZxQ,33.103460,Me & Tasty,"Thai,American (New),Breakfast & Brunch"
a5,30.202881,res_5,"Cocktail Bars,American (New),Comfort Food"
FWnV4Xuv1UP5-FGrr7aW4Q,30.202881,True Laurel,"Cocktail Bars,American (New),Comfort Food"
eOCVoXsOEnbr9c89RD0UEQ,29.498135,Pork Store Cafe,"Breakfast & Brunch,Coffee & Tea,Comfort Food"
9SsgoIxiQ7PHOfuG9P8QjQ,29.498135,Lunchpad,"Sandwiches,Comfort Food,Breakfast & Brunch"
0mNzmmh1mrdh5Cpg2QUBiw,24.405287,Lapisara Eatery,"Breakfast & Brunch,Burgers,Thai"
UGKNNQ6bjL-ZttHSdZISfA,24.405287,Blackwood,"Thai,Breakfast & Brunch,Asian Fusion"
ohWlEV89ljnbDXAHJM6_GQ,21.940345,ONE65 Bistro & Grill,"American (New),French,Bars"


In [102]:
#list of restaurants already visited by user
df_final[df_final['user_id'].isin(['B-tnUR66MKSoLtzMrHBywA'])].sort_values(by=['review_rating'], ascending=False)

,user_id,review_rating,business_id,name,categories
12,B-tnUR66MKSoLtzMrHBywA,5.0,8kck3-K4zYKTJbJko0JlXQ,Farmhouse Kitchen Thai Cuisine,"Thai,Comfort Food"
2003,B-tnUR66MKSoLtzMrHBywA,4.0,eL1aKZx9bplkBesYcGJ11w,Curbside Cafe,"Breakfast & Brunch,French,American (New)"


In [103]:
def recommender_user_to_new_restaurants(bus_id):
    
    df_predict_t = df_predict.transpose()
    #get all restaurants bus id
    
    #user predicted rating to all restaurants
    user_predicted_rating = df_predict_t[bus_id]
    user_predicted_rating = pd.DataFrame({'user_id':user_predicted_rating.index, bus_id:user_predicted_rating.values})

    
    #res already read by user
    already_visited = df_final[df_final['business_id'].isin([bus_id])]['user_id']
    
    all_rec = user_predicted_rating[~user_predicted_rating.index.isin(already_visited)]

    return all_rec.sort_values(by=[bus_id], ascending=False).iloc[0:10]

In [105]:
# List of users for whom bus_id will be recommended
user_list = recommender_user_to_new_restaurants('a1')
user_list = user_list[['user_id']]
print(user_list)

                    user_id
690  J7L5EbWEHB8njkFjZkas7w
865  NtgSafaocv1HQ7DSzfRQLw
508  Dg8phi2yU6K_CczVbZNs4w
17   0F3ziFHw3WjfZR7ngN8M7A
858  NmcfLhYfNPmck_iHYCpBeA
505  DZaKmGqgDITCAX4HVFXUuw
110  2vYEopEQ2AfwyYnbmB9faQ
273  7ArWnVr7qfZd9rStpQKyXw
738  KCUFGSlK6lX7ZSlwTbtVZQ
684  IxSNLdIqzGu8AE1mwHhTKw
